Source: http://www.rpgroup.caltech.edu/bige105/2018/code/t02_sequence_analysis_biogeography.html

In [1]:
import Bio.SeqIO
import Bio.AlignIO
import Bio.Phylo
import numpy as np
import pandas as pd

In [2]:
def compute_similarity(seq_1, seq_2):
    """
    Computes the percent similarity between two sequences ignoring gaps. 
    
    Parameters
    ----------
    seq_1, seq_2 : strings
        DNA sequences to compare. These must be the same length.
        
    Returns
    -------
    score : float
        The percent similarity between the two sequences. 
    """
    # Make sure they are the same length. 
    if len(seq_1) != len(seq_2):
        raise ValueError('Sequences must be the same length!')
        
    # Make both sequences lowercase.
    seq_1 = seq_1.lower()
    seq_2 = seq_2.lower()
        
    # Set up counters of length and similarity.
    comp_length = 0
    num_sim = 0
    
    # Iterate through each position in the sequences.
    for base in range(len(seq_1)):
        
        # Ensure we are not comparing gaps.
        if (seq_1[base] != '-') and (seq_2[base] != '-'):
            
            # Increase the counter for compared length.
            comp_length += 1
            
            # Compare the two positions.
            if seq_1[base] == seq_2[base]:
                
                # Increase the similarity counter.
                num_sim += 1
                
    # Compute and return the percent similarity.
    score = num_sim  / comp_length
    return score


def s_matrix(mat):
    '''
    Computes the S-Matrix for the Neighbor Joining algorithm. Each entry
    S_ij computes the total branch length assuming that nodes i and j form
    a new OTU.
    Parameters
    ----------
    mat : array-like. N x N matrix
        Square matrix containing similarity scores between OTUs
    Returns
    -------
    s_mat : array-like. N x N matrix
        Square matrix containing the total branch length when joining
        each possible pair of OTUs
    '''
    # Get the number of entries in matrix
    n = len(mat)
    
    # Compute the starlike length
    s_star = 1 / (n - 1) * np.sum(np.triu(mat))
    # Initialize matrix to save entries
    s_mat = np.ones_like(mat) * s_star

    # Loop through rows
    for i in range(n):
        # Loop through columns
        for j in range(n):
            # Define list of index different from i and j
            idx_out = np.arange(n)
            idx_out = np.delete(idx_out, [i, j])
            
            # To make the calculation simpler we will divide Eq. 3 into
            # the three terms
            # First term
            first_term = 1 / (2 * (n - 2)) * np.sum(mat[i, idx_out] + \
                                                    mat[j, idx_out])
            
            # Second term
            second_term = 1 / 2 * mat[i, j]
            
            # Third term
            # We have to be slightly clever for this one. To add all of the
            # terms that do not include either i or j we will eliminate
            # those columns from the matrix mat and we will use the function
            # np.triu to generate an upper triangular matrix with the lower
            # half of the matrix all zeros. That way when we add the entire
            # matrix we only sum the terms that didn't contain i or j
            reduced_mat = np.delete(mat, [i, j], axis=0)
            reduced_mat = np.delete(reduced_mat, [i, j], axis=1)
            third_term = 1 / (n - 2) * np.sum(np.triu(reduced_mat))
            
            # Add result to the s matrix
            s_mat[i, j] = first_term + second_term + third_term
            
    # Set all terms in diagonal to zero since we don't want the diagonal
    # to play any role in further steps
    np.fill_diagonal(s_mat, 0)
    
    return s_mat

def neighbor_joining(D, names=None):
    '''
    test
    '''
    # Convert distance matrix to DataFrame to track index identity
    df_D = pd.DataFrame(D)
    
    # Save original index. This is important to distinguish internal nodes
    # from external ones.
    original_idx = df_D.index.tolist()
    
    # Get shape of distance matrix
    shape = D.shape
    
    # Check that the matrix is a square matrix
    if shape[0] != shape[1]:
        raise ValueError('The ditance matrix must be square')
    
    # Check that is is larger than 2 x 2
    if shape[0] == 2:
        raise ValueError('No work to do. The tree has only two nodes')
    
    # Check that the names given are the same length as the distrance matrix
    if names == None:
        names = string.ascii_lowercase[0:len(df_D)]
    elif len(names) != len(D):
        raise ValueError('Are you sure you gave me enough names?')
    
    # Initialize dictionary to save output tree in the format 
    # 'x->y:distance'
    tree = dict()
    
    # Initialize dictionary to save the distance between recently joined
    # nodes. This is necessary to determine branch length of internal nodes
    tree_nodes = dict()
    
    # Initialize dictionary to save node names to generate the newick format
    newick_dict = dict(zip(df_D.index.tolist(), names))
    
    _loopend = shape[0]-2
    
    # Start the algorighm
    for loop in range(shape[0] - 2):
        # extract numerical values of DataFrame
        D = df_D.values
        
        print("NJ in #loop: "+str(loop)+" from "+str(_loopend))
        
        # Obtain size of distance matrix
        n = len(D)
        
        # Compute Q-matrix
        s_mat = s_matrix(D)
        
        # Find where the minimum distance is
        min_val = np.min(s_mat[np.nonzero(s_mat)])
        row, col = np.where(s_mat == min_val)
        idx = [row[0], col[0]]
        i, j = min(idx), max(idx)
        
        # Extract species index (index name)
        i_idx = df_D.iloc[i,:].name
        j_idx = df_D.iloc[j,:].name
        
        
        # Define list of index different from i and j
        idx_out = np.arange(n)
        idx_out = np.delete(idx_out, [i, j])
            
        # Compute distance of the recently joined edges
        limb_length_i = (1 / 2) * (D[i, j] + np.sum(D[i, idx_out]) / (n - 2)
                                           - np.sum(D[j, idx_out]) / (n - 2))
        limb_length_j = (1 / 2) * (D[i, j] + np.sum(D[j, idx_out]) / (n - 2)
                                           - np.sum(D[i, idx_out]) / (n - 2))
        
        # Generate expanded matrix with extra row to update distances
        mat_plus = np.zeros(np.array(D.shape) + 1)
        # Replace 0's with known distance
        mat_plus[0:D.shape[0], 0:D.shape[1]] = D
        
        # Update distance to new node
        for k in range(len(D)):
            mat_plus[k, -1] = (D[k, i] + D[k, j]) / 2
            mat_plus[-1, k] = mat_plus[k, -1]
            
        # Define new node index
        m = max(df_D.index.tolist()) + 1
        # Extract index from current DataFrame
        new_idx = df_D.index.tolist() + [m]
        # Convert extended distance matrix to data frame using same index
        df_D = pd.DataFrame(mat_plus, index=new_idx, columns=new_idx)
        # Drop the rows and columns of the two nodes that were joined
        df_D.drop(index=[i_idx, j_idx], inplace=True)
        df_D.drop(columns=[i_idx, j_idx], inplace=True)
        
        # Save the distance between the two nodes being connected. Useful to
        # determine internal branch length
        tree_nodes[m] = D[i,j]
        
        # If the node is internal (i.e. it didn't existed in the origina list)
        # update the branch length by correcting using the distance between the
        # nodes that originally formed this node.
        if i_idx not in original_idx :
            limb_length_i = limb_length_i - tree_nodes[i_idx] / 2
        if j_idx not in original_idx :
            limb_length_j = limb_length_j - tree_nodes[j_idx] / 2
        
        # Add entry to tree
        tree[str(i_idx) + '->' + str(m)] = limb_length_i
        tree[str(j_idx) + '->' + str(m)] = limb_length_j
        
        tree[str(m) + '->' + str(i_idx)] = limb_length_i
        tree[str(m) + '->' + str(j_idx)] = limb_length_j
        
        print("distance of two connected nodes: ",tree_nodes[m],". lim_length_i: ",limb_length_i,". lim_length_j: ",limb_length_j)
        
        # Newick format of this joined nodes #'({0:s}:{2:0.2f}, {1:s}:{3:0.2f})'.\
        name = '({0:s}:{2}, {1:s}:{3})'.\
               format(newick_dict[i_idx], newick_dict[j_idx],
                      limb_length_i, limb_length_j)
            
        print("THIS ",name)
            
        # Append name to newick_dict
        newick_dict[m] = name
        
    # Extract last two nodes to be joined
    last_idx = df_D.index.tolist()
    last_corr = min([tree_nodes[x] for x in last_idx])
    # Extract last distance
    last_dist = (1 / 2) * df_D.iloc[1, 0] - last_corr / 2
    
    # Append the distance of the last two-knobs left
    tree[str(last_idx[0]) + '->' + str(last_idx[1])] = last_dist
    tree[str(last_idx[1]) + '->' + str(last_idx[0])] = last_dist
    
    #'({0:s}:{2:0.2f}, {1:s}:{3:0.2f})'.\
    newick_str = '({0:s}:{2}, {1:s}:{3})'.\
               format(newick_dict[last_idx[0]], newick_dict[last_idx[1]],
                      last_dist, last_dist)
    
    return tree, newick_str

In [3]:
filename = "aligned.onlySpike.max20pIndel.noN.short600.fasta"
filepath = "files\\subsampled\\QualityCheck_Irina\\Archiv\\"+filename
aln = Bio.AlignIO.read(filepath, "fasta")
print(aln)

Alignment with 300 rows and 3821 columns
TGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGT...TAA Japan/SZ-NIG-Y223038/2022
TGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGT...TAA VNM/OUCRU-NHTD3617/2022
TGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGT...TAA KEN/C120125/2022
TGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGT...TAA Switzerland/GE-HUG-40509532/2023
TGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGT...TAA England/QEUH-326BE15A/2023
TGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGT...TAA USA/VI-CDC-2-6875014/2023
TGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGT...TAA Switzerland/BS-UHB-USB_2023_01_30_f6430886/2023
TGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGT...TAA THA/CONI-3921/2022
TGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGT...TAA USA/AK-USAFSAM-S21082/2023
TGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGT...TAA BHR/660304023/2022
TGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGT...TAA USA/UT-CDC-VSX-A079256/2023
TGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGT...TAA Canisfamiliaris/NAM/1256/2022
TGTTTGTTTTTCTTGTTTTATTG

In [4]:
"""
from IPython.display import display, clear_output

# Initialize matrix to save similarity scores
sim_mat = np.zeros([len(aln), len(aln)])

alnlen = len(aln)

# loop through species in aln
for i, sp1 in enumerate(aln):
    # Extract species 1 sequences
    seq_1 = sp1.seq
    # loop through species in aln
    for j, sp2 in enumerate(aln):
        # Extract species 2 sequences
        seq_2 = sp2.seq
        # Compute and save similarity score
        sim_mat[i, j] = 1 - compute_similarity(seq_1, seq_2)
        
        if (j%100 == 0):   #Hier eine Methode, um den Fortschritt darzustellen
            clear_output()
            percent_complete = (i*alnlen + j)/(alnlen*alnlen)
            display("i: "+str(i)+", j: "+str(j)+", von aln-length: "+str(alnlen)+" estimated-percent: "+str(percent_complete)+"%")
        
print('Similarity matrix for all {:d} sequences:'.format(len(aln)))
print(sim_mat)
"""

'\nfrom IPython.display import display, clear_output\n\n# Initialize matrix to save similarity scores\nsim_mat = np.zeros([len(aln), len(aln)])\n\nalnlen = len(aln)\n\n# loop through species in aln\nfor i, sp1 in enumerate(aln):\n    # Extract species 1 sequences\n    seq_1 = sp1.seq\n    # loop through species in aln\n    for j, sp2 in enumerate(aln):\n        # Extract species 2 sequences\n        seq_2 = sp2.seq\n        # Compute and save similarity score\n        sim_mat[i, j] = 1 - compute_similarity(seq_1, seq_2)\n        \n        if (j%100 == 0):   #Hier eine Methode, um den Fortschritt darzustellen\n            clear_output()\n            percent_complete = (i*alnlen + j)/(alnlen*alnlen)\n            display("i: "+str(i)+", j: "+str(j)+", von aln-length: "+str(alnlen)+" estimated-percent: "+str(percent_complete)+"%")\n        \nprint(\'Similarity matrix for all {:d} sequences:\'.format(len(aln)))\nprint(sim_mat)\n'

In [5]:
#Hier ist der Doppelloop um die Distanz-Matrix aufzufüllen parallelisiert
import requests
from multiprocessing.pool import ThreadPool as Pool

alnlen = len(aln)
sim_mat = np.zeros([alnlen, alnlen])

def task(idx):
    seq_1 = aln[idx].seq
    for j in range(idx,len(aln)):

        if(idx==j):
            sim_mat[idx,idx]=0
            
        if(j>idx):
            seq_2 = aln[j].seq
            
            similarity = 1 - compute_similarity(seq_1, seq_2)
            sim_mat[idx, j] = similarity
            sim_mat[j, idx] = similarity
            if ((idx%50 == 0) or (j%50==0)):
                print("computed cell (i/j): "+str((idx,j))+", similarity: "+str(similarity))
            
    return idx

    """ #Dieser Teil wurde umgeschrieben, siehe die Funktion "task". Die Symmetrie der Matrix wird jetzt ausgenutzt.
    for j, sp2 in enumerate(aln):
        seq_2 = sp2.seq
        sim_mat[idx, j] = 1 - compute_similarity(seq_1, seq_2)
        if (idx+j)%50==0:
            print("computed cell (i/j): "+str((idx,j)))
        #sim_mat[idx,1]=5
    return idx
    """

pool = Pool(20)
for result in pool.imap_unordered(task, range(0,alnlen)):
    print("_index finished: "+str(result)+" von "+str(alnlen))
    
print('Similarity matrix for all {:d} sequences:'.format(alnlen))
print(sim_mat)

computed cell (i/j): (0, 1), similarity: 0.0018406521167498857
computed cell (i/j): (0, 2), similarity: 0.0007888509071785066
computed cell (i/j): (0, 3), similarity: 0.0007888509071785066
computed cell (i/j): (0, 4), similarity: 0.0034156594850236477
computed cell (i/j): (0, 5), similarity: 0.004470155140678389
computed cell (i/j): (0, 6), similarity: 0.0005259006047857451
computed cell (i/j): (0, 7), similarity: 0.0036784025223332018
computed cell (i/j): (0, 8), similarity: 0.0042072048382855165
computed cell (i/j): (0, 9), similarity: 0.0042072048382855165
computed cell (i/j): (0, 10), similarity: 0.0042072048382855165
computed cell (i/j): (0, 11), similarity: 0.0007882291119285512
computed cell (i/j): (0, 12), similarity: 0.004217185028993153
computed cell (i/j): (0, 13), similarity: 0.0010509721492379942
computed cell (i/j): (0, 14), similarity: 0.004203888596952199
computed cell (i/j): (0, 15), similarity: 0.0
computed cell (i/j): (0, 16), similarity: 0.004470155140678389
compute

computed cell (i/j): (0, 113), similarity: 0.00026274303730955406
computed cell (i/j): (0, 114), similarity: 0.0010509721492379942
computed cell (i/j): (0, 115), similarity: 0.009458749343142392
computed cell (i/j): (0, 116), similarity: 0.008407777193904398
computed cell (i/j): (0, 117), similarity: 0.0005254860746189971
computed cell (i/j): (0, 118), similarity: 0.007894736842105288
computed cell (i/j): (0, 119), similarity: 0.0
computed cell (i/j): (0, 120), similarity: 0.008145034156594844
computed cell (i/j): (0, 121), similarity: 0.0073568050446662925
computed cell (i/j): (0, 122), similarity: 0.00788229111928529
computed cell (i/j): (0, 123), similarity: 0.0073568050446662925
computed cell (i/j): (0, 124), similarity: 0.007105263157894726
computed cell (i/j): (0, 125), similarity: 0.0036784025223332018
computed cell (i/j): (0, 126), similarity: 0.00867052023121384
computed cell (i/j): (0, 127), similarity: 0.006842105263157872
computed cell (i/j): (0, 128), similarity: 0.0005254

computed cell (i/j): (0, 245), similarity: 0.0036784025223332018
computed cell (i/j): (0, 246), similarity: 0.004470155140678389
computed cell (i/j): (10, 50), similarity: 0.0005250721974271455
computed cell (i/j): (12, 250), similarity: 0.00579862941486553
computed cell (i/j): (0, 247), similarity: 0.0005254860746189971
computed cell (i/j): (0, 248), similarity: 0.008407777193904398
computed cell (i/j): (0, 249), similarity: 0.00736842105263158
computed cell (i/j): (2, 150), similarity: 0.0010518012095713791
computed cell (i/j): (13, 250), similarity: 0.0010509721492379942
computed cell (i/j): (0, 250), similarity: 0.0015764582238571023
computed cell (i/j): (0, 251), similarity: 0.0007882291119285512
computed cell (i/j): (0, 252), similarity: 0.00736842105263158
computed cell (i/j): (1, 250), similarity: 0.0026295030239285033
computed cell (i/j): (0, 253), similarity: 0.0
computed cell (i/j): (0, 254), similarity: 0.0076195480819758465
computed cell (i/j): (0, 255), similarity: 0.0076

computed cell (i/j): (5, 200), similarity: 0.011569813305285281
computed cell (i/j): (10, 200), similarity: 0.011306863002892409
computed cell (i/j): (16, 200), similarity: 0.011306863002892409
computed cell (i/j): (32, 150), similarity: 0.0005254860746189971
computed cell (i/j): (24, 200), similarity: 0.008677359978964017
computed cell (i/j): (31, 150), similarity: 0.00026274303730955406
computed cell (i/j): (26, 200), similarity: 0.008677359978964017
computed cell (i/j): (11, 200), similarity: 0.007888509071785399
computed cell (i/j): (21, 250), similarity: 0.0018392012611665454
computed cell (i/j): (23, 250), similarity: 0.0013137151865475483
computed cell (i/j): (25, 200), similarity: 0.010780962398106775
computed cell (i/j): (24, 250), similarity: 0.0021019442984760994
computed cell (i/j): (22, 250), similarity: 0.0010509721492379942
_index finished: 23 von 300
computed cell (i/j): (26, 250), similarity: 0.0021019442984760994
computed cell (i/j): (34, 50), similarity: 0.0044701551

computed cell (i/j): (50, 71), similarity: 0.004463113678130792
computed cell (i/j): (50, 72), similarity: 0.010238907849829393
computed cell (i/j): (50, 73), similarity: 0.004200577579417164
computed cell (i/j): (50, 74), similarity: 0.004733105443071262
_index finished: 40 von 300
computed cell (i/j): (45, 250), similarity: 0.004203888596952199
computed cell (i/j): (37, 200), similarity: 0.007888509071785399
_index finished: 41 von 300
_index finished: 42 von 300
computed cell (i/j): (38, 200), similarity: 0.008151459374178271
computed cell (i/j): (50, 75), similarity: 0.004473684210526296
computed cell (i/j): (50, 76), similarity: 0.004470155140678389
computed cell (i/j): (50, 77), similarity: 0.004470155140678389
computed cell (i/j): (50, 78), similarity: 0.004996055745464134
computed cell (i/j): (50, 79), similarity: 0.01050144394854291
computed cell (i/j): (35, 250), similarity: 0.0007882291119285512
computed cell (i/j): (50, 80), similarity: 0.0007876082961407738
computed cell (

...


_index finished: 115 von 300
_index finished: 118 von 300
_index finished: 122 von 300computed cell (i/j): (180, 250), similarity: 0.004996055745464134

computed cell (i/j): (100, 132), similarity: 0.008421052631578996computed cell (i/j): (182, 200), similarity: 0.008151459374178271
_index finished: 125 von 300
_index finished: 111 von 300


computed cell (i/j): (178, 250), similarity: 0.0018392012611665454computed cell (i/j): (184, 200), similarity: 0.0034156594850236477

computed cell (i/j): (179, 250), similarity: 0.009721492380451946computed cell (i/j): (185, 200), similarity: 0.011306863002892409
_index finished: 114 von 300
computed cell (i/j): (173, 250), similarity: 0.0010509721492379942computed cell (i/j): (175, 250), similarity: 0.0036784025223332018computed cell (i/j): (183, 200), similarity: 0.00920326058374965
computed cell (i/j): (150, 159), similarity: 0.004480759093305253computed cell (i/j): (188, 200), similarity: 0.00920326058374965computed cell (i/j): (187, 200), si




computed cell (i/j): (236, 250), similarity: 0.0015764582238571023



computed cell (i/j): (100, 146), similarity: 0.0028833551769331667computed cell (i/j): (237, 250), similarity: 0.0015764582238571023
computed cell (i/j): (50, 236), similarity: 0.004470155140678389
computed cell (i/j): (234, 250), similarity: 0.0015764582238571023_index finished: 123 von 300computed cell (i/j): (150, 170), similarity: 0.003944254535892755
computed cell (i/j): (238, 250), similarity: 0.0007882291119285512


computed cell (i/j): (240, 250), similarity: 0.004466631634261642computed cell (i/j): (200, 208), similarity: 0.011043912700499647
computed cell (i/j): (241, 250), similarity: 0.009196006305832949
computed cell (i/j): (243, 250), similarity: 0.0010509721492379942
computed cell (i/j): (239, 250), similarity: 0.0013137151865475483
computed cell (i/j): (100, 147), similarity: 0.011832763607678154computed cell (i/j): (244, 250), similarity: 0.004466631634261642
computed cell (i/j): (242, 250), simil

computed cell (i/j): (100, 163), similarity: 0.0034076015727392273

computed cell (i/j): (250, 268), similarity: 0.0021019442984760994
computed cell (i/j): (200, 223), similarity: 0.008414409676571144
_index finished: 238 von 300computed cell (i/j): (100, 164), similarity: 0.0015727391874180707computed cell (i/j): (50, 254), similarity: 0.010238907849829393
computed cell (i/j): (250, 269), similarity: 0.005784906652642641

_index finished: 239 von 300
_index finished: 251 von 300
_index finished: 242 von 300
_index finished: 256 von 300
_index finished: 244 von 300
_index finished: 252 von 300
_index finished: 258 von 300
_index finished: 249 von 300
_index finished: 261 von 300
_index finished: 255 von 300
computed cell (i/j): (150, 187), similarity: 0.0005254860746189971

computed cell (i/j): (50, 255), similarity: 0.010238907849829393
computed cell (i/j): (250, 270), similarity: 0.0013147515119642517
computed cell (i/j): (150, 188), similarity: 0.0007882291119285512
computed cell (i

computed cell (i/j): (50, 271), similarity: 0.004470155140678389


computed cell (i/j): (100, 186), similarity: 0.008421052631578996
computed cell (i/j): (200, 241), similarity: 0.0020986358866736943
computed cell (i/j): (250, 297), similarity: 0.009729161188535396computed cell (i/j): (50, 272), similarity: 0.004733105443071262
computed cell (i/j): (100, 187), similarity: 0.008684210526315739
computed cell (i/j): (200, 242), similarity: 0.009466210886142523
computed cell (i/j): (150, 209), similarity: 0.0005254860746189971

computed cell (i/j): (200, 243), similarity: 0.008677359978964017
computed cell (i/j): (100, 188), similarity: 0.008947368421052593
computed cell (i/j): (250, 298), similarity: 0.004203888596952199
computed cell (i/j): (50, 273), similarity: 0.004470155140678389
computed cell (i/j): (150, 210), similarity: 0.0007882291119285512computed cell (i/j): (250, 299), similarity: 0.0015764582238571023

computed cell (i/j): (200, 244), similarity: 0.010780962398106775
compute


computed cell (i/j): (200, 271), similarity: 0.008151459374178271
computed cell (i/j): (150, 241), similarity: 0.00788229111928529
computed cell (i/j): (100, 220), similarity: 0.007894736842105288computed cell (i/j): (200, 272), similarity: 0.008414409676571144

computed cell (i/j): (150, 242), similarity: 0.0010509721492379942
computed cell (i/j): (200, 273), similarity: 0.008151459374178271
computed cell (i/j): (100, 221), similarity: 0.0028833551769331667
computed cell (i/j): (200, 274), similarity: 0.008151459374178271
computed cell (i/j): (150, 243), similarity: 0.0007882291119285512
computed cell (i/j): (200, 275), similarity: 0.0023609653725078505
computed cell (i/j): (100, 222), similarity: 0.008145034156594844
computed cell (i/j): (200, 276), similarity: 0.008151459374178271computed cell (i/j): (150, 244), similarity: 0.0034156594850236477

computed cell (i/j): (150, 245), similarity: 0.0034156594850236477
computed cell (i/j): (100, 223), similarity: 0.008157894736842142compu

In [6]:
np.savetxt(filename+"_dm.txt", sim_mat)

In [7]:
species_names = [sp.name for sp in aln]
tree_dict, newick = neighbor_joining(sim_mat, species_names)

NJ in #loop: 0 from 298
distance of two connected nodes:  0.0005234231876471984 . lim_length_i:  -7.668245216117114e-07 . lim_length_j:  0.0005241900121688101
THIS  (BRA/DF-NXBS126GENOV827515027588/2021:-7.668245216117114e-07, VEN/Ara6800/2021:0.0005241900121688101)
NJ in #loop: 1 from 298
distance of two connected nodes:  0.0007851347814707976 . lim_length_i:  0.0004212221528967757 . lim_length_j:  0.0001022010347504227
THIS  (PER/AFI00996/2021:0.0004212221528967757, (BRA/DF-NXBS126GENOV827515027588/2021:-7.668245216117114e-07, VEN/Ara6800/2021:0.0005241900121688101):0.0001022010347504227)
NJ in #loop: 2 from 298
distance of two connected nodes:  0.0007869884575025798 . lim_length_i:  0.0002474316516475287 . lim_length_j:  0.0005395568058550512
THIS  (TGO/nmimr-TG-SARS-CoV-2-TGS-047-21/2021:0.0002474316516475287, MLI/MAL0134/2021:0.0005395568058550512)
NJ in #loop: 3 from 298
distance of two connected nodes:  0.0005246589716684236 . lim_length_i:  0.0002614381288769483 . lim_length_j:

distance of two connected nodes:  0.001308557969117996 . lim_length_i:  0.0008061023953202318 . lim_length_j:  0.00024074397997416504
THIS  (HongKong/HKPU-00126/2021:0.0008061023953202318, (ESP/HUD-79063186/2020:1.937002979339464e-06, England/CAMC-B51982/2020:0.000521486184667859):0.00024074397997416504)
NJ in #loop: 25 from 298
distance of two connected nodes:  0.0018319811567651945 . lim_length_i:  0.0007847201241441846 . lim_length_j:  0.00104726103262101
THIS  (ETH/AH-003/2020:0.0007847201241441846, OU471436:0.00104726103262101)
NJ in #loop: 26 from 298
distance of two connected nodes:  0.0014394137660297956 . lim_length_i:  0.000522798914944075 . lim_length_j:  6.24272703123388e-07
THIS  (Estonia/Cov0667/2020:0.000522798914944075, (ETH/AH-003/2020:0.0007847201241441846, OU471436:0.00104726103262101):6.24272703123388e-07)
NJ in #loop: 27 from 298
distance of two connected nodes:  0.0010468463752943968 . lim_length_i:  0.00026108501753116346 . lim_length_j:  6.265762924361797e-07
TH

distance of two connected nodes:  0.0002704012365872734 . lim_length_i:  0.000261627575776132 . lim_length_j:  8.401804746783863e-08
THIS  (DJI/NAMRU3_CL1/2020:0.000261627575776132, (USA/CA-CZB-29279/2021:-2.8691922901360328e-08, (RUS/-42/2020:-5.7165656272475246e-08, (NZL/20VR2917/2020:-1.1389823939382537e-07, (USA/MD-MDH-0029/2020:-2.2693686943693614e-07, (ZAF/SCV2-GS-0421/2020:-4.521674466214168e-07, (TUN/TUN_ADAGE_4681/2020:0.00026165859689000065, (IRQ/USAFSAM-S085/2020:0.00026160599545590676, (USA/MS-CDC-LC0004694/2021:0.0002615011822061884, (England/MILK-800BDA/2020:0.0002612923291933538, ((SRB/KV0359-25-07/2020:-1.9592924432832934e-07, (NZL/20CV0055/2020:-3.905118959109284e-07, England/NORT-29E6D6/2020:0.00026210210571951014):1.9592924432832934e-07):0.00026108501753116346, (Estonia/Cov0667/2020:0.000522798914944075, (ETH/AH-003/2020:0.0007847201241441846, OU471436:0.00104726103262101):6.24272703123388e-07):6.265762924361797e-07):4.1926463024586844e-07):2.1041161741078285e-07):1.

distance of two connected nodes:  0.0014934315972700782 . lim_length_i:  9.193719483922488e-05 . lim_length_j:  3.8918602072574504e-05
THIS  ((HongKong/HKPU-00126/2021:0.0008061023953202318, (ESP/HUD-79063186/2020:1.937002979339464e-06, England/CAMC-B51982/2020:0.000521486184667859):0.00024074397997416504):9.193719483922488e-05, (OD916317:0.0007482184065318673, (((USA/MP-CDC-2-3831022/2020:0.0005233011870406502, USA/TX-DSHS-0913/2020:1.2200060654820113e-07):0.00026114972054552734, ((USA/VA-VTVAS3-GSC23203/2021:0.0005242015367000093, MAR/Morocco_HMIMV_279CC/2020:-7.783490528112842e-07):0.00026160033426557514, (DJI/NAMRU3_CL1/2020:0.000261627575776132, (USA/CA-CZB-29279/2021:-2.8691922901360328e-08, (RUS/-42/2020:-5.7165656272475246e-08, (NZL/20VR2917/2020:-1.1389823939382537e-07, (USA/MD-MDH-0029/2020:-2.2693686943693614e-07, (ZAF/SCV2-GS-0421/2020:-4.521674466214168e-07, (TUN/TUN_ADAGE_4681/2020:0.00026165859689000065, (IRQ/USAFSAM-S085/2020:0.00026160599545590676, (USA/MS-CDC-LC000469

distance of two connected nodes:  0.0018127789062693412 . lim_length_i:  0.0002158324074488724 . lim_length_j:  4.589125457808707e-05
THIS  ((BTN/RCDC0001-SV210017/2020:0.0010648906341149227, NZL/20CV0600/2020:0.0007670905226502718):0.0002158324074488724, (RUS/Primorye_3551/2020:0.0004420191207149633, ((HongKong/HKPU-00126/2021:0.0008061023953202318, (ESP/HUD-79063186/2020:1.937002979339464e-06, England/CAMC-B51982/2020:0.000521486184667859):0.00024074397997416504):9.193719483922488e-05, (OD916317:0.0007482184065318673, (((USA/MP-CDC-2-3831022/2020:0.0005233011870406502, USA/TX-DSHS-0913/2020:1.2200060654820113e-07):0.00026114972054552734, ((USA/VA-VTVAS3-GSC23203/2021:0.0005242015367000093, MAR/Morocco_HMIMV_279CC/2020:-7.783490528112842e-07):0.00026160033426557514, (DJI/NAMRU3_CL1/2020:0.000261627575776132, (USA/CA-CZB-29279/2021:-2.8691922901360328e-08, (RUS/-42/2020:-5.7165656272475246e-08, (NZL/20VR2917/2020:-1.1389823939382537e-07, (USA/MD-MDH-0029/2020:-2.2693686943693614e-07, (

...

distance of two connected nodes:  5.450803076442381e-05 . lim_length_i:  -1.4321457201301856e-09 . lim_length_j:  1.432145720136962e-09
THIS  (USA/OSPHL07521/2022:-1.4321457201301856e-09, (Switzerland/ZH-ETHZ-38004464/2022:-2.788915349691539e-09, ((Switzerland/BE-SRO-5000138287/2022:-5.5574268851112454e-09, (LAO/LOMWRU-0842/2022:0.0005254415905321024, (USA/ND-CDC-2-6543571/2022:0.0002626262454013926, ((USA/MA-MGB-09214/2022:0.00039438388942566386, (FRA/2208V240427/2022:0.00026274303730949855, FRA/2211V180257/2022:0.00026274303730949855):0.00013110218519338875):0.0001310804491343519, (VNM/OUCRU-NHTD3606/2022:-6.539142244468333e-07, ((OX370327:0.0002627430373094985, USA/MA-CDCBI-CRSP_7KKAVYBGH75DTURV/2022:0.0002627430373094985):0.00025879459769831425, (((SYC/SS10817/2022:0.00027180552440158896, Switzerland/AG-ETHZ-38005168/2022:0.00025368055021740814):0.00013377401808862958, ((OP926913:0.00026780729758020983, OQ155260:0.000521043609598297):0.00025582979025569243, (LAO/LOMWRU-0651/2022:0.

distance of two connected nodes:  6.8135038455529765e-06 . lim_length_i:  -1.9481393985520436e-10 . lim_length_j:  1.9481393987638018e-10
THIS  (Liechtenstein/FL-Risch-1018R14206/2022:-1.9481393985520436e-10, (Liechtenstein/FL-Risch-1006R14232/2022:-3.784956546044795e-10, (BGD/icddrb-TND-04-1538/2022:-7.359637728616194e-10, (USA/OSPHL07521/2022:-1.4321457201301856e-09, (Switzerland/ZH-ETHZ-38004464/2022:-2.788915349691539e-09, ((Switzerland/BE-SRO-5000138287/2022:-5.5574268851112454e-09, (LAO/LOMWRU-0842/2022:0.0005254415905321024, (USA/ND-CDC-2-6543571/2022:0.0002626262454013926, ((USA/MA-MGB-09214/2022:0.00039438388942566386, (FRA/2208V240427/2022:0.00026274303730949855, FRA/2211V180257/2022:0.00026274303730949855):0.00013110218519338875):0.0001310804491343519, (VNM/OUCRU-NHTD3606/2022:-6.539142244468333e-07, ((OX370327:0.0002627430373094985, USA/MA-CDCBI-CRSP_7KKAVYBGH75DTURV/2022:0.0002627430373094985):0.00025879459769831425, (((SYC/SS10817/2022:0.00027180552440158896, Switzerland/

distance of two connected nodes:  4.2584399034706103e-07 . lim_length_i:  -1.379932076267e-11 . lim_length_j:  1.3799320736888435e-11
THIS  (JPN/JP-33-USAFSAM-S20233/2022:-1.379932076267e-11, (Switzerland/BE-Risch-1006R13952/2022:-2.6708362707876174e-11, (USA/MN-MDH-30171/2022:-5.17474527854736e-11, (VNM/OUCRU-NHTD3541/2022:-1.0035869631532836e-10, (Liechtenstein/FL-Risch-1018R14206/2022:-1.9481393985520436e-10, (Liechtenstein/FL-Risch-1006R14232/2022:-3.784956546044795e-10, (BGD/icddrb-TND-04-1538/2022:-7.359637728616194e-10, (USA/OSPHL07521/2022:-1.4321457201301856e-09, (Switzerland/ZH-ETHZ-38004464/2022:-2.788915349691539e-09, ((Switzerland/BE-SRO-5000138287/2022:-5.5574268851112454e-09, (LAO/LOMWRU-0842/2022:0.0005254415905321024, (USA/ND-CDC-2-6543571/2022:0.0002626262454013926, ((USA/MA-MGB-09214/2022:0.00039438388942566386, (FRA/2208V240427/2022:0.00026274303730949855, FRA/2211V180257/2022:0.00026274303730949855):0.00013110218519338875):0.0001310804491343519, (VNM/OUCRU-NHTD3606

distance of two connected nodes:  6.653812349172829e-09 . lim_length_i:  -2.695179691404592e-13 . lim_length_j:  2.6951798652531416e-13
THIS  (England/QEUH-32584859/2022:-2.695179691404592e-13, (Liechtenstein/FL-Risch-0926R13832/2022:-5.174745452019708e-13, (OX431955:-9.951433123725484e-13, (Liechtenstein/FL-Risch-0926R13833/2022:-1.9165723424540565e-12, (USA/KY-CDC-LC0897655/2022:-3.6962466507667324e-12, (USA/CT-CDC-LC0910577/2022:-7.137579671181657e-12, (JPN/JP-33-USAFSAM-S20233/2022:-1.379932076267e-11, (Switzerland/BE-Risch-1006R13952/2022:-2.6708362707876174e-11, (USA/MN-MDH-30171/2022:-5.17474527854736e-11, (VNM/OUCRU-NHTD3541/2022:-1.0035869631532836e-10, (Liechtenstein/FL-Risch-1018R14206/2022:-1.9481393985520436e-10, (Liechtenstein/FL-Risch-1006R14232/2022:-3.784956546044795e-10, (BGD/icddrb-TND-04-1538/2022:-7.359637728616194e-10, (USA/OSPHL07521/2022:-1.4321457201301856e-09, (Switzerland/ZH-ETHZ-38004464/2022:-2.788915349691539e-09, ((Switzerland/BE-SRO-5000138287/2022:-5.55

distance of two connected nodes:  0.0005249729078733861 . lim_length_i:  0.0002627430373094306 . lim_length_j:  6.78168458889683e-17
THIS  (VNM/OUCRU-NHTD3537/2022:0.0002627430373094306, (THA/MTM_01_416/2022:0.00026274303730937257, (OX403776:0.0002627430373092632, (Japan/SZ-NIG-Y221831/2022:0.0002627430373090575, (DJI/NAMRU3_H274/2022:0.00026274303730866827, (JPN/JP-33-USAFSAM-S20116/2022:0.00026274303730793036, (Liechtenstein/FL-Risch-0923R13838/2022:0.00026274303730652713, (Liechtenstein/FL-Risch-0926R13825/2022:0.0002627430373038528, (LAO/LOMWRU-0853/2022:0.0002627430372987448, (England/PLYM-32585CA7/2022:0.00026274303726363214, (Liechtenstein/FL-Risch-0922R13836/2022:-1.4061809442550421e-13, (England/QEUH-32584859/2022:-2.695179691404592e-13, (Liechtenstein/FL-Risch-0926R13832/2022:-5.174745452019708e-13, (OX431955:-9.951433123725484e-13, (Liechtenstein/FL-Risch-0926R13833/2022:-1.9165723424540565e-12, (USA/KY-CDC-LC0897655/2022:-3.6962466507667324e-12, (USA/CT-CDC-LC0910577/2022:-

In [8]:
from io import StringIO
tree = Bio.Phylo.read(StringIO(newick), 'newick')
Bio.Phylo.draw_ascii(tree)

 , JPN/JP-33-USAFSAM-S20307/2022
 |
 | JPN/JP-33-USAFSAM-S20119/2022
 |
 , SYC/SS10820/2022
 |
 |_ USA/CA-CDC-STM-QCEZ3MZYB/2022
 |
 , VNM/OUCRU-NHTD3527/2022
 |
 |_ OX384661
_|
 , USA/IN-CDC-STM-EUAZBUKTD/2022
 |
 , Japan/SZ-NIG-Y222446/2022
 |
 | KEN/C120167/2022
 |
 , Japan/SZ-NIG-Y222013/2022
 |
 , Switzerland/TI-EOC-38004704/2022
 |
 , USA/AL-CDC-STM-MQW9GSG7B/2022
 |
 | USA/MA-CDCBI-CRSP_TL4HBIWJRAMWLKSI/2022
 |
 , Japan/SZ-NIG-Y223300/2023
 |
 , KEN/SS10826/2022
 |
 , Japan/SZ-NIG-Y222635/2022
 |
 , VNM/OUCRU-NHTD3537/2022
 |
 , THA/MTM_01_416/2022
 |
 , OX403776
 |
 , Japan/SZ-NIG-Y221831/2022
 |
 , DJI/NAMRU3_H274/2022
 |
 , JPN/JP-33-USAFSAM-S20116/2022
 |
 , Liechtenstein/FL-Risch-0923R13838/2022
 |
 , Liechtenstein/FL-Risch-0926R13825/2022
 |
 , LAO/LOMWRU-0853/2022
 |
 , England/PLYM-32585CA7/2022
 |
 , Liechtenstein/FL-Risch-0922R13836/2022
 |
 , England/QEUH-32584859/2022
 |
 , Liechtenstein/FL-Risch-0926R13832/2022
 |
 , OX431955
 |
 , Liechtenstein/FL-Risch-0926R13833/

In [9]:
### Den phylogenetischen Baum exportieren

Bio.Phylo.write(tree, filename+"_NJtree.xml", "phyloxml")

#Dann auf https://itol.embl.de/ via upload einlesen
#Ergebnis: ....

1

In [10]:
tree.total_branch_length()

0.10808111070511506